In [44]:
import pandas as pd
import numpy as np

In [45]:
rainfall    =  pd.read_csv('Hoppers Crossing-Hourly-Rainfall.csv')
river_level =  pd.read_csv('Hoppers Crossing-Hourly-River-Level.csv')

In [46]:
relation = pd.merge(rainfall,river_level,on='Date/Time')

In [47]:
relation["Level (m)"].mean()

1.4537013637805263

In [49]:
relation = relation.iloc[::-1]

In [50]:
relation.index = relation.index[::-1]

In [51]:
relation

,Date/Time,Current rainfall (mm),Cumulative rainfall (mm),Level (m)
0,2009-12-02 00:00:00,0.0,0.0,1.40
1,2009-12-02 01:00:00,0.0,0.0,1.40
2,2009-12-02 02:00:00,0.0,0.0,1.40
3,2009-12-02 03:00:00,0.0,0.0,1.39
4,2009-12-02 04:00:00,0.0,0.0,1.39
...,...,...,...,...
78820,2018-12-02 13:00:00,0.0,0.0,1.63
78821,2018-12-02 14:00:00,0.0,0.0,1.63
78822,2018-12-02 15:00:00,0.0,0.0,1.62
78823,2018-12-02 16:00:00,0.0,0.0,1.61


In [53]:
relation['date'] = pd.to_datetime(relation['Date/Time'])

In [59]:
relation.resample('1M', on = 'date').mean()

,Current rainfall (mm),Cumulative rainfall (mm),Level (m)
date,,,
2009-12-31,0.050000,0.634444,1.385625
2010-01-31,0.010753,0.251075,1.319570
2010-02-28,0.047619,0.914881,1.332619
2010-03-31,0.067204,1.050269,1.368548
2010-04-30,0.050000,0.336389,1.397653
...,...,...,...
2018-08-31,0.040323,0.366667,1.527016
2018-09-30,0.000000,0.155556,1.500917
2018-10-31,0.026991,0.515520,1.516937


In [97]:
monthly_avg_rainfall = dict()
monthly_avg_cumulative_rainfall = dict()
monthly_avg_Level = dict()

In [98]:
#Briefly after this step when program get one random day it ll know which month it is and 
# what is average water level for this season
for month in range(1,12+1):
    count = 0
    value_rainfall = 0
    value_level = 0
    value_cumulative = 0
    
    
    if month == 12:
        month = 0
        
    for i in range(relation.resample('1M', on = 'date').mean().shape[0]):   
        
        if i%12 == month:
            
            count +=1 
            value_rainfall += relation.resample('1M', on = 'date').mean().iloc[i][0]
            value_cumulative += relation.resample('1M', on = 'date').mean().iloc[i][1]
            value_level += relation.resample('1M', on = 'date').mean().iloc[i][2]
        
    
    value_rainfall  = value_rainfall/count
    value_cumulative = value_cumulative/count
    value_level = value_level/count
    
    if month == 0:
        month = 12
    
    monthly_avg_rainfall[month] = value_rainfall
    monthly_avg_cumulative_rainfall[month] = value_cumulative
    monthly_avg_Level[month] = value_level
    print('this is count ',count,'Month num ',month)   
            
            
            

this is count  9 Month num  1
this is count  9 Month num  2
this is count  9 Month num  3
this is count  9 Month num  4
this is count  9 Month num  5
this is count  9 Month num  6
this is count  9 Month num  7
this is count  9 Month num  8
this is count  9 Month num  9
this is count  9 Month num  10
this is count  9 Month num  11
this is count  10 Month num  12


In [116]:
relation.resample('1D', on = 'date').mean().shape

(3288, 3)

In [157]:
input_all_batches = list()
output_all_batches = list()

In [158]:
# Max limit height of water in river
max_height = 1.5

In [159]:
# with this step program process data in a form that we want to give as an input to NN

#input for NN ( Water level height of river, average height for this season, Avg rainfall next week , Avg rainfall second week
#ouyput -> Max height in 3rd week

for i in range(relation.resample('1D', on = 'date').mean().shape[0] - 21 ): 
    one_input = list()
    output = list()
    print(relation.resample('1D', on = 'date').mean().iloc[i])
    #inputs
    level = relation.resample('1D', on = 'date').mean().iloc[1][2]
    month = relation.resample('1D', on = 'date').mean().index[i].month
    
    month_avg = monthly_avg_Level[month]
    
    cumulative_rain_fall_1 = relation.resample('1D', on = 'date').mean().iloc[i:i+7,1].mean()
    cumulative_rain_fall_2 = relation.resample('1D', on = 'date').mean().iloc[i+7:i+14,1].mean()
    
    one_input.append(level)
    one_input.append(month_avg)
    one_input.append(cumulative_rain_fall_1)
    one_input.append(cumulative_rain_fall_2)
    
    #output
    max_level = relation.resample('1D', on = 'date').mean().iloc[i+14:i+21,2].max()
    output_all_batches.append(max_level)
    
    input_all_batches.append(one_input)
input_all_batches

Current rainfall (mm)       0.000
Cumulative rainfall (mm)    0.450
Level (m)                   1.395
Name: 2009-12-02 00:00:00, dtype: float64
Current rainfall (mm)       0.0000
Cumulative rainfall (mm)    1.3500
Level (m)                   1.3775
Name: 2009-12-03 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.359583
Name: 2009-12-04 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.357083
Name: 2009-12-05 00:00:00, dtype: float64
Current rainfall (mm)       0.00000
Cumulative rainfall (mm)    0.00000
Level (m)                   1.34875
Name: 2009-12-06 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.349167
Name: 2009-12-07 00:00:00, dtype: float64
Current rainfall (mm)       0.666667
Cumulative rainfall (mm)    3.708333
Level (m)                 

Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.247083
Name: 2010-01-28 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.239167
Name: 2010-01-29 00:00:00, dtype: float64
Current rainfall (mm)       0.000
Cumulative rainfall (mm)    0.000
Level (m)                   1.225
Name: 2010-01-30 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.050000
Level (m)                   1.214167
Name: 2010-01-31 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.075000
Level (m)                   1.199167
Name: 2010-02-01 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.181667
Name: 2010-02-02 00:00:00, dtype: float64
Current rainfall (mm)       0.00000
Cumulative rainfall (mm)    0.00000
Level (m)          

Current rainfall (mm)       0.00
Cumulative rainfall (mm)    0.00
Level (m)                   1.32
Name: 2010-03-24 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.316667
Name: 2010-03-25 00:00:00, dtype: float64
Current rainfall (mm)       0.00
Cumulative rainfall (mm)    0.00
Level (m)                   1.31
Name: 2010-03-26 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.306667
Name: 2010-03-27 00:00:00, dtype: float64
Current rainfall (mm)       0.0
Cumulative rainfall (mm)    0.1
Level (m)                   1.3
Name: 2010-03-28 00:00:00, dtype: float64
Current rainfall (mm)       0.083333
Cumulative rainfall (mm)    2.775000
Level (m)                   1.437500
Name: 2010-03-29 00:00:00, dtype: float64
Current rainfall (mm)       0.00
Cumulative rainfall (mm)    1.20
Level (m)                   1.46
Name: 2010-03-30 00:00

Name: 2010-05-20 00:00:00, dtype: float64
Current rainfall (mm)       0.00
Cumulative rainfall (mm)    0.00
Level (m)                   1.41
Name: 2010-05-21 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.424583
Name: 2010-05-22 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.405417
Name: 2010-05-23 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.458333
Level (m)                   1.421667
Name: 2010-05-24 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    1.983333
Level (m)                   1.445833
Name: 2010-05-25 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.483333
Level (m)                   1.445000
Name: 2010-05-26 00:00:00, dtype: float64
Current rainfall (mm)       0.0000
Cumulative rainfa

Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.429167
Name: 2010-07-17 00:00:00, dtype: float64
Current rainfall (mm)       0.250000
Cumulative rainfall (mm)    1.033333
Level (m)                   1.434583
Name: 2010-07-18 00:00:00, dtype: float64
Current rainfall (mm)       0.000
Cumulative rainfall (mm)    1.425
Level (m)                   1.450
Name: 2010-07-19 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.083333
Level (m)                   1.437917
Name: 2010-07-20 00:00:00, dtype: float64
Current rainfall (mm)       0.000
Cumulative rainfall (mm)    0.075
Level (m)                   1.425
Name: 2010-07-21 00:00:00, dtype: float64
Current rainfall (mm)       0.083333
Cumulative rainfall (mm)    0.116667
Level (m)                   1.423750
Name: 2010-07-22 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.208333
Level (m)                 

...

Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.532083
Name: 2018-05-14 00:00:00, dtype: float64
Current rainfall (mm)       0.083333
Cumulative rainfall (mm)    0.000000
Level (m)                   1.530000
Name: 2018-05-15 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.391667
Level (m)                   1.560833
Name: 2018-05-16 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.535417
Name: 2018-05-17 00:00:00, dtype: float64
Current rainfall (mm)       0.0000
Cumulative rainfall (mm)    0.0000
Level (m)                   1.5025
Name: 2018-05-18 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.500833
Name: 2018-05-19 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.133333
Level (m)     

Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.534167
Name: 2018-07-09 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.489167
Name: 2018-07-10 00:00:00, dtype: float64
Current rainfall (mm)       0.083333
Cumulative rainfall (mm)    0.441667
Level (m)                   1.523750
Name: 2018-07-11 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.666667
Level (m)                   1.545417
Name: 2018-07-12 00:00:00, dtype: float64
Current rainfall (mm)       0.000
Cumulative rainfall (mm)    0.000
Level (m)                   1.515
Name: 2018-07-13 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.484167
Name: 2018-07-14 00:00:00, dtype: float64
Current rainfall (mm)       0.00000
Cumulative rainfall (mm)    0.00000
Level (m)          

Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.575000
Level (m)                   1.527917
Name: 2018-09-02 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.083333
Level (m)                   1.515417
Name: 2018-09-03 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.504583
Name: 2018-09-04 00:00:00, dtype: float64
Current rainfall (mm)       0.0
Cumulative rainfall (mm)    0.0
Level (m)                   1.5
Name: 2018-09-05 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.083333
Level (m)                   1.500000
Name: 2018-09-06 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.083333
Level (m)                   1.500000
Name: 2018-09-07 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)              

Current rainfall (mm)       0.00
Cumulative rainfall (mm)    0.00
Level (m)                   1.49
Name: 2018-10-27 00:00:00, dtype: float64
Current rainfall (mm)       0.00000
Cumulative rainfall (mm)    0.00000
Level (m)                   1.48875
Name: 2018-10-28 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.000000
Level (m)                   1.482174
Name: 2018-10-29 00:00:00, dtype: float64
Current rainfall (mm)       0.00000
Cumulative rainfall (mm)    0.00000
Level (m)                   1.48375
Name: 2018-10-30 00:00:00, dtype: float64
Current rainfall (mm)       0.000
Cumulative rainfall (mm)    0.000
Level (m)                   1.485
Name: 2018-10-31 00:00:00, dtype: float64
Current rainfall (mm)       0.00
Cumulative rainfall (mm)    0.00
Level (m)                   1.49
Name: 2018-11-01 00:00:00, dtype: float64
Current rainfall (mm)       0.000000
Cumulative rainfall (mm)    0.008333
Level (m)                   1.489583
Name: 2018

[[1.3775000000000002,
  1.45757243343574,
  0.7869047619047619,
  0.47619047619047616],
 [1.3775000000000002,
  1.45757243343574,
  1.0654761904761905,
  0.13333333333333333],
 [1.3775000000000002, 1.45757243343574, 0.930952380952381, 0.4440476190476191],
 [1.3775000000000002, 1.45757243343574, 1.005952380952381, 0.8404761904761905],
 [1.3775000000000002, 1.45757243343574, 1.005952380952381, 0.8404761904761905],
 [1.3775000000000002, 1.45757243343574, 1.005952380952381, 0.8404761904761905],
 [1.3775000000000002, 1.45757243343574, 1.005952380952381, 0.8404761904761905],
 [1.3775000000000002,
  1.45757243343574,
  0.47619047619047616,
  0.8404761904761905],
 [1.3775000000000002,
  1.45757243343574,
  0.13333333333333333,
  0.8404761904761905],
 [1.3775000000000002, 1.45757243343574, 0.4440476190476191, 0.675],
 [1.3775000000000002,
  1.45757243343574,
  0.8404761904761905,
  0.5678571428571428],
 [1.3775000000000002,
  1.45757243343574,
  0.8404761904761905,
  0.5678571428571428],
 [1.37

In [164]:
output_all_batches = np.array(output_all_batches)
input_all_batches = np.array(input_all_batches)

In [167]:
input_all_batches.shape

(3267, 4)

In [168]:
output_all_batches = output_all_batches.reshape(3267,1)
input_all_batches = input_all_batches.reshape(3267,4)

In [169]:
input_all_batches.shape,output_all_batches.shape

((3267, 4), (3267, 1))

In [170]:
input_all_batches = input_all_batches.T
output_all_batches = output_all_batches.T
input_all_batches.shape,output_all_batches.shape

((4, 3267), (1, 3267))

In [198]:
#Save the new data
np.savetxt('input.csv',input_all_batches,delimiter=',')
np.savetxt('output.csv',output_all_batches,delimiter=',')